# Ejercicio 10

### Imports

In [ ]:
#!/usr/bin/python3
# Importing python3 from local, just use "python3 <binary>" if is not the same location

# /
# ** Luis ROSARIO, 2021
# ** Exercise 7 Software
# ** File description:
# ** Media matcher (moovies)
# ** https://github.com/Luisrosario2604
# */

# Imports
import os
from multiprocessing import Pool
import imghdr
import time
import cv2

valid_images = [".jpg", ".gif", ".png", ".tga", ".jpeg"]

### Configurable variables

In [2]:
query = "./media/query/LGE.jpg" #@param {type:"string"}
covers = "./media/covers/" #@param {type:"string"}

### Checking the arguments

In [3]:
def check_arguments(query, covers):
    

    if not os.path.exists(query):
        print("[ERROR] Original image should be an image")
        exit(84)

    type = imghdr.what(query)

    if type not in ['gif', 'jpeg', 'png', 'tiff', 'bmp']:
        print("[ERROR] Original image should be an image")
        exit(84)

    if not os.path.isdir(covers):
        print("[ERROR] Image collection should be an existing directory")
        exit(84)

    return query, covers

In [4]:
query, covers = check_arguments(query, covers)

### Getting the images of the second arguments (folder)

In [5]:
def get_images(input_dir):
    images = []

    for path in os.listdir(input_dir):
        ext = os.path.splitext(path)[1]
        if ext.lower() not in valid_images:
            continue

        filename = os.path.basename(path)
        filename = filename[::-1]
        filename = filename[filename.find('.') + 1:]
        filename = filename[::-1]

        images.append([os.path.join(input_dir, path), filename, ext])

    if not images:
        print("[ERROR] Not even one image found in the input directory")
        exit(84)

    return images

In [6]:
images_covers = get_images(covers)

### Detection of the query with the images

In [7]:
def sift_detection(image):
    img1 = cv2.imread(query, cv2.IMREAD_GRAYSCALE)  # queryImage
    img2 = cv2.imread(image, cv2.IMREAD_GRAYSCALE)  # trainImage
    # Initiate SIFT detector
    sift = cv2.SIFT_create()
    # find the keypoints and descriptors with SIFT
    kp1, des1 = sift.detectAndCompute(img1, None)
    kp2, des2 = sift.detectAndCompute(img2, None)
    # BFMatcher with default params
    bf = cv2.BFMatcher()
    matches = bf.knnMatch(des1, des2, k=2)
    # Apply ratio test
    good = []
    for m, n in matches:
        if m.distance < 0.75 * n.distance:
            good.append([m])
    # cv22.drawMatchesKnn expects list of lists as matches.
    img3 = cv2.drawMatchesKnn(img1, kp1, img2, kp2, good, None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)

    return [len(good), img3] 

### With multiprocessing Pool

In [8]:
start = time.time()

images_covers_names = []
for i in images_covers:
    images_covers_names.append(i[0])

pool = Pool() # Create a multiprocessing Pool
results = pool.map(sift_detection, images_covers_names)


end = time.time()
print("Elapsed (with compilation) = %s" % (end - start))

Elapsed (with compilation) = 1.4219388961791992


### Without multiprocessing Pool

In [9]:
start = time.time()

result = []
for i in images_covers:
    result.append(sift_detection(i[0]))
    
end = time.time()

print("Elapsed (with compilation) = %s" % (end - start))

Elapsed (with compilation) = 1.667222023010254


### Result

In [10]:
minimum_score = 40
best_match = None
best_score = 0

for r in result:
    if r[0] >= best_score and r[0] >= minimum_score:
        best_match = r[1]
        best_score = r[0]

if best_score == 0:
    print("None of the covers found ! Should have more than 40 matches !")
else:
    cv2.imshow('Number of matches : ' + str(best_score), best_match)

    while True:
        key = cv2.waitKey(1)
        if key == ord('q'):
            break

    cv2.destroyAllWindows()

### I tried with 250 images 

##### Results :     
###### 64.4s with multiprocessing Pool
######  98.2s without multiprocessing Pool
    